In [ ]:
elastic_rods_dir = '../../../elastic_rods/python/'
weaving_dir = '../../'
import os
import os.path as osp
import sys; sys.path.append(elastic_rods_dir); sys.path.append(weaving_dir)
import numpy as np, elastic_rods, linkage_vis
import numpy.linalg as la
from bending_validation import suppress_stdout as so
import matplotlib.pyplot as plt
from elastic_rods import EnergyType, InterleavingType

# weaving
import analysis_helper, ribbon_linkage_helper, mesh_vis, linkage_utils, compute_curve_from_curvature, pipeline_helper, importlib
importlib.reload(analysis_helper)
importlib.reload(ribbon_linkage_helper)
importlib.reload(mesh_vis)
importlib.reload(linkage_utils)
importlib.reload(compute_curve_from_curvature)
importlib.reload(pipeline_helper)
from analysis_helper import (compare_turning_angle,
                            is_on_sphere, 
                            get_distance_to_center_scalar_field, 
                            plot_curvatures, 
                            get_curvature_scalar_field,
                            construct_elastic_rod_loop_from_rod_segments, 
                            concatenate_rod_properties_from_rod_segments, 
                            compute_min_distance_rigid_transformation)
from ribbon_linkage_helper import (update_rest_curvature, 
                                   set_ribbon_linkage,
                                   export_linkage_geometry_to_obj,
                                   write_linkage_ribbon_output_florin)
from compute_curve_from_curvature import (match_geo_curvature_and_edge_len, get_all_curve_pattern)
from linkage_utils import order_segments_by_ribbons, get_turning_angle_and_length_from_ordered_rods

from pipeline_helper import (initialize_linkage, get_normal_deviation, set_joint_vector_field, stage_1_optimization, initialize_stage_2_optimizer, stage_2_optimization, InputOrganizer, write_all_output, set_surface_view_options, get_structure_analysis_view, get_double_side_view, show_selected_joints, get_fixed_boundary_joint)
import vis.fields
import matplotlib.cm as cm
import time

In [ ]:
# rod_length = curved_linkage.totalRestLength() / curved_linkage.numSegments()
rod_length = 0.079785061039255
width = rod_length / 25 * 5
thickness = width / 5 * 0.5
print(width, thickness)
scale = 1
io = InputOrganizer('bunny_head_small_triaxial_{}'.format(scale), thickness, width, weaving_dir)

In [ ]:
OPTS = elastic_rods.NewtonOptimizerOptions()
OPTS.gradTol = 1e-6
OPTS.verbose = 1;
OPTS.beta = 1e-8
OPTS.niter = 200
OPTS.verboseNonPosDef = False
rw = 0.01
sw = 0.01

In [ ]:
with so(): curved_linkage = initialize_linkage(surface_path = io.SURFACE_PATH, useCenterline = True, model_path = io.MODEL_PATH, cross_section = io.RIBBON_CS, subdivision_res = io.SUBDIVISION_RESOLUTION, interleaving_type=InterleavingType.weaving)
curved_linkage.set_design_parameter_config(use_restLen = True, use_restKappa = True)
curved_save_tgt_joint_pos = curved_linkage.jointPositions();
curved_linkage_view = get_double_side_view(curved_linkage, flip = False)
curved_linkage_view.show()

In [ ]:
input_joint_normals = np.reshape(curved_linkage.get_closest_point_normal(curved_linkage.jointPositions()), (curved_linkage.numJoints(), 3))
current_joint_normals = [curved_linkage.joint(i).normal for i in range(curved_linkage.numJoints())]
current_joint_normals[35] = input_joint_normals[35]
with so(): curved_linkage = initialize_linkage(surface_path = io.SURFACE_PATH, useCenterline = True, model_path = io.MODEL_PATH, cross_section = io.RIBBON_CS, subdivision_res = io.SUBDIVISION_RESOLUTION, use_constant_width=True, interleaving_type=InterleavingType.weaving, input_joint_normals = current_joint_normals)
curved_linkage.set_design_parameter_config(use_restLen = True, use_restKappa = True)
curved_save_tgt_joint_pos = curved_linkage.jointPositions();
curved_linkage_view = linkage_vis.LinkageViewer(curved_linkage)
# set_surface_view_options(curved_linkage_view)
curved_linkage_view.show()

In [ ]:
curved_linkage.setExtendedDoFsPSRL(np.load('bunny_head_small_triaxial_1_dof.npy'))

In [ ]:
topColor =[180/255., 158/255., 119/255.]
bottomColor =[79/255., 158/255., 246/255.]
heights = curved_linkage.visualizationGeometryHeightColors()
colors = np.take(np.array([topColor, bottomColor]), heights < heights.mean(), axis=0)

In [ ]:
curved_linkage.attraction_weight = 1e-5

In [ ]:
def eqm_callback(prob, i):
    curved_linkage_view.update(scalarField = colors)

In [ ]:
elastic_rods.compute_equilibrium(curved_linkage, callback = eqm_callback, options = OPTS, fixedVars = [])

In [ ]:
def visualize_crossing(index, linkage, view):
    vf = show_selected_joints(linkage, [index], flip = False)
    view.update(vectorField = vf, scalarField = colors)

In [ ]:
ribbons = order_segments_by_ribbons(curved_linkage)
_, _, _, _, all_joint_index, _ = get_turning_angle_and_length_from_ordered_rods(ribbons, curved_linkage, rest = True)

def get_ribbon_crossing_list(index, linkage, view):
    selected_list = []
    selected_ribbon = []
    for ribbon_index, index_list in enumerate(all_joint_index):
        if index in set(index_list):
            selected_ribbon.append(ribbon_index)
            selected_list.append(index_list)
    print("The crossing {} belongs to ribbon {}".format(index, ', '.join([str(x) for x in selected_ribbon])))
    for i in range(len(selected_list)):
        print('Ribbon {}: {}'.format(selected_ribbon[i], selected_list[i]))

In [ ]:
def select_crossing(crossing_index):
    visualize_crossing(crossing_index, curved_linkage, curved_linkage_view)
    get_ribbon_crossing_list(crossing_index, curved_linkage, curved_linkage_view)

selV =13
select_crossing(selV)


In [ ]:
vf = show_selected_joints(curved_linkage, [selV], flip = True)
curved_linkage_view.update(vectorField = vf, scalarField = colors)
curved_linkage_view.show()